In [1]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow import keras
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framewo

In [2]:
df = pd.read_csv("dataset/train.csv")
df.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [3]:
trainPath = "dataset/Train Images/"
Images = os.listdir(trainPath)
len(Images)

5983

In [4]:
len(df)

5983

In [5]:
df.isna().sum()

Image    0
Class    0
dtype: int64

In [6]:
temp = []

for i in range(len(df)):
    img = cv2.imread(trainPath + Images[i])
    img = np.resize(img, (60, 60, 3))
    img = img.astype('float')
    img = img/255.0
    
    temp.append({"Image" : Images[i], "imageData" : img})

temp = pd.DataFrame(temp)
temp.head()

,Image,imageData
0,image5402.jpg,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
1,image9707.jpg,"[[[0.7333333333333333, 0.7529411764705882, 0.7..."
2,image10325.jpg,"[[[0.09411764705882353, 0.043137254901960784, ..."
3,image3662.jpg,"[[[0.3764705882352941, 0.592156862745098, 0.10..."
4,image2709.jpg,"[[[0.2549019607843137, 0.2627450980392157, 0.2..."


In [7]:
df = pd.merge(df, temp, on = "Image")
df.head()

,Image,Class,imageData
0,image7042.jpg,Food,"[[[0.17647058823529413, 0.17647058823529413, 0..."
1,image3327.jpg,misc,"[[[0.38823529411764707, 0.47058823529411764, 0..."
2,image10335.jpg,Attire,"[[[0.027450980392156862, 0.058823529411764705,..."
3,image8019.jpg,Food,"[[[0.10588235294117647, 0.16862745098039217, 0..."
4,image2128.jpg,Attire,"[[[0.6627450980392157, 0.6549019607843137, 0.6..."


In [8]:
le = LabelEncoder()
df["Class"] = le.fit_transform(df.Class)
df.head()

,Image,Class,imageData
0,image7042.jpg,2,"[[[0.17647058823529413, 0.17647058823529413, 0..."
1,image3327.jpg,3,"[[[0.38823529411764707, 0.47058823529411764, 0..."
2,image10335.jpg,0,"[[[0.027450980392156862, 0.058823529411764705,..."
3,image8019.jpg,2,"[[[0.10588235294117647, 0.16862745098039217, 0..."
4,image2128.jpg,0,"[[[0.6627450980392157, 0.6549019607843137, 0.6..."


In [9]:
df.Class.unique()

array([2, 3, 0, 1])

In [10]:
le.inverse_transform(df.Class)

array(['Food', 'misc', 'Attire', ..., 'Food', 'Food', 'Attire'],
      dtype=object)

In [11]:
x = []
y = []

for i in range(len(df)):
    x.append(df.imageData[i])
    y.append(df.Class[i])
    
x = np.array(x)
y = np.array(y)
y = to_categorical(y)

In [15]:
trainX, testX, trainY, testY = train_test_split(x,y, test_size = 0.1, random_state = 42)

In [16]:
def buildModel():
    model = Sequential()
    
    model.add(Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(60, 60, 3)))
    model.add(MaxPool2D(pool_size=(1,1)))
    
    model.add(Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
    model.add(MaxPool2D(pool_size=(1,1)))
    
    model.add(Flatten())
    
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(4, activation= 'softmax'))
    
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [17]:
model = buildModel()
model.fit(trainX, trainY, epochs=100, verbose=1, validation_split=0.1)

Train on 4845 samples, validate on 539 samples
Epoch 1/100
4845/4845 [==============================] - ETA: 1:32 - loss: 1.3862 - accuracy: 0.25 - ETA: 1:02 - loss: 1.3770 - accuracy: 0.31 - ETA: 52s - loss: 1.3655 - accuracy: 0.3438 - ETA: 47s - loss: 1.3929 - accuracy: 0.320 - ETA: 44s - loss: 1.4085 - accuracy: 0.293 - ETA: 42s - loss: 1.4075 - accuracy: 0.276 - ETA: 41s - loss: 1.4014 - accuracy: 0.267 - ETA: 39s - loss: 1.3934 - accuracy: 0.289 - ETA: 38s - loss: 1.3888 - accuracy: 0.291 - ETA: 39s - loss: 1.3854 - accuracy: 0.293 - ETA: 40s - loss: 1.3830 - accuracy: 0.301 - ETA: 40s - loss: 1.3764 - accuracy: 0.320 - ETA: 39s - loss: 1.3814 - accuracy: 0.305 - ETA: 39s - loss: 1.3784 - accuracy: 0.299 - ETA: 38s - loss: 1.3833 - accuracy: 0.291 - ETA: 37s - loss: 1.3824 - accuracy: 0.293 - ETA: 37s - loss: 1.3811 - accuracy: 0.286 - ETA: 36s - loss: 1.3804 - accuracy: 0.286 - ETA: 36s - loss: 1.3785 - accuracy: 0.281 - ETA: 35s - loss: 1.3781 - accuracy: 0.282 - ETA: 35s - loss

4845/4845 [==============================] - ETA: 30s - loss: 1.1674 - accuracy: 0.531 - ETA: 31s - loss: 1.2083 - accuracy: 0.468 - ETA: 31s - loss: 1.1753 - accuracy: 0.479 - ETA: 31s - loss: 1.1746 - accuracy: 0.468 - ETA: 31s - loss: 1.1517 - accuracy: 0.475 - ETA: 30s - loss: 1.1755 - accuracy: 0.458 - ETA: 30s - loss: 1.1717 - accuracy: 0.459 - ETA: 30s - loss: 1.1802 - accuracy: 0.425 - ETA: 30s - loss: 1.1751 - accuracy: 0.427 - ETA: 30s - loss: 1.1760 - accuracy: 0.437 - ETA: 29s - loss: 1.1786 - accuracy: 0.443 - ETA: 29s - loss: 1.1814 - accuracy: 0.442 - ETA: 30s - loss: 1.1838 - accuracy: 0.435 - ETA: 30s - loss: 1.1854 - accuracy: 0.433 - ETA: 31s - loss: 1.1869 - accuracy: 0.439 - ETA: 31s - loss: 1.1888 - accuracy: 0.433 - ETA: 31s - loss: 1.1930 - accuracy: 0.433 - ETA: 32s - loss: 1.2022 - accuracy: 0.423 - ETA: 33s - loss: 1.2032 - accuracy: 0.424 - ETA: 33s - loss: 1.2027 - accuracy: 0.421 - ETA: 33s - loss: 1.2052 - accuracy: 0.419 - ETA: 33s - loss: 1.2004 - accur

4845/4845 [==============================] - ETA: 38s - loss: 1.1796 - accuracy: 0.375 - ETA: 36s - loss: 1.1218 - accuracy: 0.437 - ETA: 36s - loss: 1.1598 - accuracy: 0.406 - ETA: 35s - loss: 1.1675 - accuracy: 0.421 - ETA: 35s - loss: 1.1485 - accuracy: 0.456 - ETA: 35s - loss: 1.1528 - accuracy: 0.458 - ETA: 35s - loss: 1.1566 - accuracy: 0.450 - ETA: 34s - loss: 1.1535 - accuracy: 0.453 - ETA: 34s - loss: 1.1611 - accuracy: 0.454 - ETA: 34s - loss: 1.1588 - accuracy: 0.459 - ETA: 34s - loss: 1.1614 - accuracy: 0.457 - ETA: 34s - loss: 1.1585 - accuracy: 0.450 - ETA: 34s - loss: 1.1470 - accuracy: 0.456 - ETA: 34s - loss: 1.1500 - accuracy: 0.455 - ETA: 34s - loss: 1.1439 - accuracy: 0.458 - ETA: 35s - loss: 1.1518 - accuracy: 0.451 - ETA: 35s - loss: 1.1446 - accuracy: 0.457 - ETA: 34s - loss: 1.1452 - accuracy: 0.465 - ETA: 34s - loss: 1.1413 - accuracy: 0.473 - ETA: 34s - loss: 1.1459 - accuracy: 0.470 - ETA: 34s - loss: 1.1357 - accuracy: 0.477 - ETA: 33s - loss: 1.1340 - accur

4845/4845 [==============================] - ETA: 1:03 - loss: 1.0008 - accuracy: 0.62 - ETA: 1:05 - loss: 1.0418 - accuracy: 0.56 - ETA: 1:05 - loss: 1.0478 - accuracy: 0.54 - ETA: 1:05 - loss: 1.0380 - accuracy: 0.54 - ETA: 1:05 - loss: 1.0137 - accuracy: 0.55 - ETA: 1:04 - loss: 1.0159 - accuracy: 0.55 - ETA: 1:04 - loss: 1.0072 - accuracy: 0.56 - ETA: 1:01 - loss: 1.0058 - accuracy: 0.57 - ETA: 58s - loss: 0.9969 - accuracy: 0.5799 - ETA: 55s - loss: 0.9804 - accuracy: 0.590 - ETA: 53s - loss: 0.9708 - accuracy: 0.593 - ETA: 50s - loss: 0.9949 - accuracy: 0.578 - ETA: 49s - loss: 1.0070 - accuracy: 0.567 - ETA: 47s - loss: 1.0039 - accuracy: 0.564 - ETA: 45s - loss: 1.0023 - accuracy: 0.562 - ETA: 44s - loss: 0.9989 - accuracy: 0.568 - ETA: 43s - loss: 1.0005 - accuracy: 0.566 - ETA: 43s - loss: 1.0147 - accuracy: 0.557 - ETA: 42s - loss: 1.0066 - accuracy: 0.559 - ETA: 41s - loss: 1.0094 - accuracy: 0.548 - ETA: 40s - loss: 1.0126 - accuracy: 0.546 - ETA: 39s - loss: 1.0071 - accu

4845/4845 [==============================] - ETA: 36s - loss: 0.7120 - accuracy: 0.781 - ETA: 34s - loss: 0.7232 - accuracy: 0.734 - ETA: 34s - loss: 0.7489 - accuracy: 0.729 - ETA: 33s - loss: 0.7151 - accuracy: 0.757 - ETA: 34s - loss: 0.7100 - accuracy: 0.750 - ETA: 34s - loss: 0.6745 - accuracy: 0.765 - ETA: 34s - loss: 0.6686 - accuracy: 0.763 - ETA: 34s - loss: 0.6648 - accuracy: 0.777 - ETA: 33s - loss: 0.6835 - accuracy: 0.763 - ETA: 33s - loss: 0.7050 - accuracy: 0.753 - ETA: 32s - loss: 0.6850 - accuracy: 0.761 - ETA: 32s - loss: 0.7021 - accuracy: 0.752 - ETA: 31s - loss: 0.7164 - accuracy: 0.747 - ETA: 31s - loss: 0.7149 - accuracy: 0.745 - ETA: 31s - loss: 0.7250 - accuracy: 0.737 - ETA: 31s - loss: 0.7168 - accuracy: 0.734 - ETA: 30s - loss: 0.7105 - accuracy: 0.739 - ETA: 30s - loss: 0.7193 - accuracy: 0.736 - ETA: 30s - loss: 0.7106 - accuracy: 0.738 - ETA: 30s - loss: 0.7246 - accuracy: 0.731 - ETA: 29s - loss: 0.7275 - accuracy: 0.730 - ETA: 30s - loss: 0.7316 - accur

...

4845/4845 [==============================] - ETA: 40s - loss: 3.0223e-05 - accuracy: 1.000 - ETA: 41s - loss: 2.8256e-05 - accuracy: 1.000 - ETA: 40s - loss: 3.3789e-05 - accuracy: 1.000 - ETA: 40s - loss: 3.4788e-05 - accuracy: 1.000 - ETA: 40s - loss: 4.3088e-05 - accuracy: 1.000 - ETA: 39s - loss: 5.1807e-05 - accuracy: 1.000 - ETA: 39s - loss: 1.0352e-04 - accuracy: 1.000 - ETA: 38s - loss: 9.5462e-05 - accuracy: 1.000 - ETA: 37s - loss: 0.0013 - accuracy: 1.0000    - ETA: 37s - loss: 0.0012 - accuracy: 1.000 - ETA: 36s - loss: 0.0019 - accuracy: 1.000 - ETA: 36s - loss: 0.0018 - accuracy: 1.000 - ETA: 35s - loss: 0.0016 - accuracy: 1.000 - ETA: 35s - loss: 0.0015 - accuracy: 1.000 - ETA: 35s - loss: 0.0014 - accuracy: 1.000 - ETA: 34s - loss: 0.0013 - accuracy: 1.000 - ETA: 34s - loss: 0.0016 - accuracy: 1.000 - ETA: 34s - loss: 0.0015 - accuracy: 1.000 - ETA: 34s - loss: 0.0032 - accuracy: 1.000 - ETA: 33s - loss: 0.0030 - accuracy: 1.000 - ETA: 33s - loss: 0.0029 - accuracy: 1.0

4845/4845 [==============================] - ETA: 37s - loss: 1.6200e-05 - accuracy: 1.000 - ETA: 36s - loss: 1.1105e-05 - accuracy: 1.000 - ETA: 37s - loss: 0.0184 - accuracy: 0.9896    - ETA: 37s - loss: 0.0147 - accuracy: 0.992 - ETA: 36s - loss: 0.0118 - accuracy: 0.993 - ETA: 36s - loss: 0.0098 - accuracy: 0.994 - ETA: 36s - loss: 0.0084 - accuracy: 0.995 - ETA: 36s - loss: 0.0074 - accuracy: 0.996 - ETA: 35s - loss: 0.0065 - accuracy: 0.996 - ETA: 35s - loss: 0.0059 - accuracy: 0.996 - ETA: 35s - loss: 0.0054 - accuracy: 0.997 - ETA: 35s - loss: 0.0049 - accuracy: 0.997 - ETA: 35s - loss: 0.0046 - accuracy: 0.997 - ETA: 35s - loss: 0.0042 - accuracy: 0.997 - ETA: 34s - loss: 0.0040 - accuracy: 0.997 - ETA: 34s - loss: 0.0054 - accuracy: 0.996 - ETA: 34s - loss: 0.0051 - accuracy: 0.996 - ETA: 34s - loss: 0.0052 - accuracy: 0.996 - ETA: 33s - loss: 0.0049 - accuracy: 0.996 - ETA: 33s - loss: 0.0047 - accuracy: 0.996 - ETA: 33s - loss: 0.0054 - accuracy: 0.997 - ETA: 33s - loss: 0.

4845/4845 [==============================] - ETA: 52s - loss: 0.2670 - accuracy: 0.968 - ETA: 47s - loss: 0.1431 - accuracy: 0.984 - ETA: 45s - loss: 0.1020 - accuracy: 0.989 - ETA: 45s - loss: 0.0807 - accuracy: 0.992 - ETA: 44s - loss: 0.0713 - accuracy: 0.987 - ETA: 44s - loss: 0.0602 - accuracy: 0.989 - ETA: 43s - loss: 0.0645 - accuracy: 0.986 - ETA: 44s - loss: 0.0629 - accuracy: 0.988 - ETA: 44s - loss: 0.0702 - accuracy: 0.986 - ETA: 43s - loss: 0.0653 - accuracy: 0.987 - ETA: 43s - loss: 0.0778 - accuracy: 0.985 - ETA: 42s - loss: 0.0732 - accuracy: 0.987 - ETA: 42s - loss: 0.0692 - accuracy: 0.988 - ETA: 42s - loss: 0.0755 - accuracy: 0.986 - ETA: 41s - loss: 0.0740 - accuracy: 0.985 - ETA: 41s - loss: 0.0724 - accuracy: 0.986 - ETA: 40s - loss: 0.0756 - accuracy: 0.985 - ETA: 39s - loss: 0.1015 - accuracy: 0.980 - ETA: 39s - loss: 0.1183 - accuracy: 0.978 - ETA: 38s - loss: 0.1553 - accuracy: 0.976 - ETA: 38s - loss: 0.1597 - accuracy: 0.971 - ETA: 37s - loss: 0.1689 - accur

4845/4845 [==============================] - ETA: 39s - loss: 0.0232 - accuracy: 1.000 - ETA: 38s - loss: 0.0770 - accuracy: 0.968 - ETA: 38s - loss: 0.0642 - accuracy: 0.968 - ETA: 38s - loss: 0.0487 - accuracy: 0.976 - ETA: 37s - loss: 0.0419 - accuracy: 0.981 - ETA: 37s - loss: 0.0392 - accuracy: 0.984 - ETA: 36s - loss: 0.0338 - accuracy: 0.986 - ETA: 36s - loss: 0.0340 - accuracy: 0.988 - ETA: 36s - loss: 0.0305 - accuracy: 0.989 - ETA: 36s - loss: 0.0277 - accuracy: 0.990 - ETA: 36s - loss: 0.0510 - accuracy: 0.988 - ETA: 36s - loss: 0.0469 - accuracy: 0.989 - ETA: 36s - loss: 0.0434 - accuracy: 0.990 - ETA: 37s - loss: 0.0404 - accuracy: 0.991 - ETA: 38s - loss: 0.0378 - accuracy: 0.991 - ETA: 38s - loss: 0.0356 - accuracy: 0.992 - ETA: 38s - loss: 0.0340 - accuracy: 0.992 - ETA: 38s - loss: 0.0324 - accuracy: 0.993 - ETA: 37s - loss: 0.0309 - accuracy: 0.993 - ETA: 37s - loss: 0.0307 - accuracy: 0.993 - ETA: 36s - loss: 0.0294 - accuracy: 0.994 - ETA: 36s - loss: 0.0281 - accur

4845/4845 [==============================] - ETA: 39s - loss: 0.0022 - accuracy: 1.000 - ETA: 39s - loss: 0.0131 - accuracy: 1.000 - ETA: 39s - loss: 0.0160 - accuracy: 1.000 - ETA: 39s - loss: 0.0368 - accuracy: 0.992 - ETA: 39s - loss: 0.0364 - accuracy: 0.993 - ETA: 38s - loss: 0.0416 - accuracy: 0.984 - ETA: 38s - loss: 0.0573 - accuracy: 0.977 - ETA: 40s - loss: 0.0512 - accuracy: 0.980 - ETA: 43s - loss: 0.0457 - accuracy: 0.982 - ETA: 43s - loss: 0.0447 - accuracy: 0.981 - ETA: 42s - loss: 0.0433 - accuracy: 0.983 - ETA: 44s - loss: 0.0439 - accuracy: 0.981 - ETA: 43s - loss: 0.0417 - accuracy: 0.983 - ETA: 42s - loss: 0.0390 - accuracy: 0.984 - ETA: 42s - loss: 0.0375 - accuracy: 0.985 - ETA: 41s - loss: 0.0353 - accuracy: 0.986 - ETA: 40s - loss: 0.0363 - accuracy: 0.985 - ETA: 40s - loss: 0.0363 - accuracy: 0.986 - ETA: 39s - loss: 0.0423 - accuracy: 0.983 - ETA: 39s - loss: 0.0419 - accuracy: 0.982 - ETA: 38s - loss: 0.0404 - accuracy: 0.983 - ETA: 38s - loss: 0.0395 - accur

In [18]:
_, accuracy = model.evaluate(testX, testY, verbose=1)
accuracy

599/599 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 2ms/step


0.39065107703208923

In [19]:
np.argmax(model.predict(testX), axis=1)

array([2, 2, 2, 3, 3, 2, 0, 0, 2, 2, 2, 3, 3, 2, 1, 0, 2, 0, 1, 3, 2, 2,
       0, 2, 2, 1, 3, 1, 3, 1, 0, 2, 0, 3, 0, 2, 1, 2, 3, 2, 2, 0, 0, 1,
       3, 3, 0, 2, 1, 2, 2, 0, 2, 2, 2, 3, 0, 0, 1, 0, 2, 2, 2, 0, 2, 2,
       3, 2, 0, 0, 2, 0, 0, 3, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 3, 2, 0,
       3, 3, 2, 2, 3, 0, 2, 3, 2, 2, 2, 2, 0, 2, 3, 3, 2, 2, 2, 2, 0, 2,
       0, 2, 3, 2, 2, 2, 0, 3, 3, 0, 2, 0, 2, 0, 2, 2, 2, 2, 0, 3, 2, 3,
       0, 0, 3, 2, 2, 0, 0, 0, 3, 1, 2, 0, 2, 0, 0, 0, 3, 2, 2, 3, 3, 0,
       2, 3, 2, 2, 0, 2, 2, 1, 2, 2, 3, 0, 2, 2, 2, 0, 2, 2, 3, 1, 0, 2,
       3, 0, 2, 2, 3, 2, 2, 0, 3, 0, 2, 0, 2, 3, 2, 0, 0, 0, 3, 0, 0, 2,
       0, 2, 2, 1, 0, 3, 0, 2, 2, 2, 2, 3, 2, 0, 2, 0, 0, 0, 0, 0, 2, 3,
       2, 0, 0, 2, 3, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 3, 1, 2, 2,
       3, 0, 3, 3, 2, 0, 0, 2, 2, 0, 3, 0, 1, 3, 0, 0, 2, 2, 2, 2, 2, 2,
       0, 2, 0, 0, 0, 0, 2, 1, 3, 2, 2, 3, 0, 3, 2, 2, 0, 3, 0, 2, 0, 2,
       3, 0, 0, 2, 2, 3, 2, 0, 2, 0, 0, 2, 0, 2, 0,

In [20]:
np.argmax(testY, axis=1)

array([0, 2, 2, 3, 2, 2, 1, 0, 2, 2, 2, 3, 2, 1, 3, 2, 2, 0, 0, 2, 3, 2,
       3, 2, 2, 2, 3, 3, 1, 3, 2, 2, 0, 0, 0, 2, 1, 0, 3, 3, 3, 0, 2, 2,
       3, 2, 3, 1, 1, 1, 1, 3, 2, 2, 0, 2, 3, 2, 2, 2, 3, 2, 2, 0, 2, 0,
       2, 2, 0, 2, 2, 0, 0, 0, 3, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 3, 2, 1,
       3, 2, 0, 0, 3, 0, 2, 1, 2, 2, 1, 2, 0, 2, 2, 2, 3, 0, 2, 0, 2, 2,
       3, 3, 3, 0, 3, 1, 0, 2, 2, 1, 2, 2, 2, 2, 3, 3, 1, 0, 2, 1, 2, 3,
       2, 0, 0, 2, 1, 3, 3, 2, 2, 2, 3, 0, 3, 3, 0, 2, 2, 2, 0, 2, 0, 2,
       2, 0, 0, 2, 2, 2, 3, 3, 2, 2, 3, 0, 0, 2, 2, 2, 2, 3, 2, 0, 2, 2,
       2, 2, 2, 2, 3, 3, 0, 3, 2, 2, 2, 0, 3, 2, 1, 2, 0, 3, 0, 2, 0, 2,
       2, 2, 2, 3, 0, 1, 0, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 0,
       2, 0, 2, 2, 1, 2, 1, 3, 3, 2, 2, 0, 0, 3, 2, 2, 2, 1, 2, 1, 1, 3,
       2, 2, 0, 3, 2, 0, 3, 3, 2, 0, 1, 2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 3,
       0, 0, 1, 2, 0, 1, 2, 1, 2, 2, 1, 2, 3, 2, 0, 2, 3, 2, 2, 3, 0, 0,
       3, 0, 3, 3, 2, 1, 3, 0, 3, 1, 2, 3, 3, 1, 3,

In [21]:
model.save_weights("weights/model.h5")

In [22]:
df = pd.read_csv("dataset/test.csv")
df.head()

,Image
0,image6245.jpg
1,image10409.jpg
2,image8692.jpg
3,image10517.jpg
4,image2580.jpg


In [24]:
testPath = "dataset/Test Images/"
Images = os.listdir(testPath)
len(Images)

3219

In [25]:
df.isna().sum()

Image    0
dtype: int64

In [26]:
temp = []

for i in range(len(df)):
    img = cv2.imread(testPath + Images[i])
    img = np.resize(img, (60, 60, 3))
    img = img.astype('float')
    img = img/255.0
    
    temp.append({"Image" : Images[i], "imageData" : img})

temp = pd.DataFrame(temp)
temp.head()

,Image,imageData
0,image5761.jpg,"[[[0.043137254901960784, 0.11372549019607843, ..."
1,image7487.jpg,"[[[0.2980392156862745, 0.34509803921568627, 0...."
2,image4596.jpg,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
3,image8532.jpg,"[[[0.9098039215686274, 0.6901960784313725, 0.3..."
4,image1449.jpg,"[[[0.5529411764705883, 0.596078431372549, 0.67..."


In [27]:
df = pd.merge(df, temp, on = "Image")
df.head()

,Image,imageData
0,image6245.jpg,"[[[0.9921568627450981, 0.984313725490196, 0.98..."
1,image10409.jpg,"[[[0.0, 0.0, 0.011764705882352941], [0.0, 0.0,..."
2,image8692.jpg,"[[[0.3333333333333333, 0.36470588235294116, 0...."
3,image10517.jpg,"[[[0.4, 0.17647058823529413, 0.207843137254901..."
4,image2580.jpg,"[[[0.34901960784313724, 0.34901960784313724, 0..."


In [28]:
x = []

for i in range(len(df)):
    x.append(df.imageData[i])
    
x = np.array(x)

In [29]:
predict = np.argmax(model.predict(x), axis=1)

In [30]:
predict

array([2, 2, 2, ..., 2, 3, 1])

In [32]:
predict = le.inverse_transform(predict)
predict

array(['Food', 'Food', 'Food', ..., 'Food', 'misc',
       'Decorationandsignage'], dtype=object)

In [33]:
df["Class"] = predict

In [34]:
df.head()

,Image,imageData,Class
0,image6245.jpg,"[[[0.9921568627450981, 0.984313725490196, 0.98...",Food
1,image10409.jpg,"[[[0.0, 0.0, 0.011764705882352941], [0.0, 0.0,...",Food
2,image8692.jpg,"[[[0.3333333333333333, 0.36470588235294116, 0....",Food
3,image10517.jpg,"[[[0.4, 0.17647058823529413, 0.207843137254901...",Food
4,image2580.jpg,"[[[0.34901960784313724, 0.34901960784313724, 0...",Food


In [35]:
df = df.drop(['imageData'], axis=1)

In [36]:
df.head()

,Image,Class
0,image6245.jpg,Food
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Food


In [37]:
df.to_csv("output/outputCNN.csv")